Make an OTU table from DADA sequences and metadata

In [1]:
import re
import operator as op
from itertools import zip_longest, repeat

import numpy as np
import pandas as pd
import skbio
from biom import Table
from biom.util import biom_open
from Bio import SeqIO
from fn import F

Parse taxonomy and remove entries lacking a minimum of `MINRANK` classified taxonomic ranks

In [2]:
MINRANK = 6

parse_rank = (
    F(re.compile('\([0-9.]+\)$').sub, '') >>  # remove confidence scores
    (re.compile('isolate').sub, '') >> # remove the 'isolate' postfix
    (lambda x: x.replace('_', ' ')) >> 
    str.strip
)

def parse_taxa(minrank: int, tax: str):
    parsed = [parse_rank(rank) for rank in tax.split(';')][1:7]
    return (parsed + ['unknown'] * 6)[:6] if len(parsed) >= minrank else np.nan

taxonomy = pd.read_csv(
    'dada/taxonomy.tsv', sep='\t', index_col=0, header=None, names=['seqid', 'tax'],
    converters={'tax': F(parse_taxa, MINRANK)}
)
taxonomy_complete = taxonomy[~taxonomy['tax'].isna()]
classified_entries = set(taxonomy_complete.index)

f'{len(classified_entries)} entries left out of {len(taxonomy)}'

'216 entries left out of 282'

In [3]:
# sample metadata
metadata = pd.read_csv('metadata.tsv', sep='\t').set_index('sample', drop=False)

# phylogenetic tree
with open('tree/tree.nwk') as buffer:
    tree = skbio.read(buffer, format="newick", into=skbio.tree.TreeNode)

# DADA ASV counts and taxonomy
counts = pd.read_csv('dada/counts.tsv', sep='\t', index_col=0)

# remove reference sequences from the tree, subset classified and inserted tips
inserted_tips = [tip.name for tip in tree.tips() if tip.name in classified_entries]
tree_inserted = tree.shear(inserted_tips)
counts_inserted = counts.loc[inserted_tips]

f'{len(counts_inserted)} sequences were both classified and inserted into the reference tree'

'216 sequences were both classified and inserted into the reference tree'

Make an OTU table

In [4]:
data = counts_inserted.values
samples = counts_inserted.keys()
observations = counts_inserted.index
metadata_keys = ['cycle', 'replicate']

sample_metadata = [
    row.to_dict() for _, row in metadata.loc[samples, metadata_keys].iterrows()
]
observation_metadata = [{'taxonomy': tax} for tax in taxonomy_complete.loc[observations, 'tax']]


table = Table(data, observations, samples, observation_metadata, sample_metadata, table_id='table')

Export the OTU table and the sheared tree

In [5]:
! mkdir -p stats

with biom_open('stats/table.biom', 'w') as out:  
    table.to_hdf5(out, 'table')
    
skbio.write(tree_inserted, 'newick', 'stats/tree.nwk')

'stats/tree.nwk'